In [24]:
import requests
from bs4 import BeautifulSoup
import re


In [54]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Search results sections
url = 'https://www.zonaprop.com.ar/casas-departamentos-ph-alquiler-caballito.html'

# Listing section
url = 'https://www.zonaprop.com.ar/propiedades/clasificado/alclapin-colpayo-700-53876211.html'

all_listings = []


response = requests.get(url, headers=headers)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')

# soup

## Extract Listings

In [3]:
listings = soup.find_all('div', class_='PostingCardLayout-sc-i1odl-0')
listings

In [30]:
# soup.find('a', class_='PageItem-sc-n5babu-1')

next_page = soup.find('a', attrs={'data-qa': 'PAGING_2'})
f"https://www.zonaprop.com.ar{next_page['href']}" if next_page else None

'https://www.zonaprop.com.ar/casas-departamentos-ph-alquiler-caballito-pagina-2.html'

In [18]:
listings[0]

<div class="PostingCardLayout-sc-i1odl-0 egwEUc" data-id="53972741" data-posting-type="PROPERTY" data-qa="posting PROPERTY" data-to-posting="/propiedades/clasificado/alclapin-alquiler-departamento-2-ambientes-en-caballito-53972741.html"><div class="PostingCardContainer-sc-i1odl-1 kfOVKt"><div class="GalleryContainer-sc-n2cjqs-0 ckhZnU" data-qa="POSTING_CARD_GALLERY"><div class="lazyload-wrapper" style="width:100%;height:100%;position:absolute"><img alt="Departamento · 42m² · 2 Ambientes · Alquiler Departamento 2 Ambientes en Caballito" fetchpriority="high" height="100%" loading="eager" src="https://imgar.zonapropcdn.com/avisos/1/00/53/97/27/41/360x266/1918959678.jpg?isFirstImage=true" style="object-fit:cover" width="100%"/></div></div><div class="PostingContainer-sc-i1odl-2 iQlPeD"><div class="PostingTop-sc-i1odl-3 hCsHEP"><div><div class="PostingCardRow-sc-i1odl-5 kgqRDY"><div class="PostingPricesAndPublisher-sc-i1odl-8 hcxUmI"><div class="PostingCardPriceBlock-sc-12dh9kl-0 diTcpX"><d

## Extract listing details

In [87]:
feature_section = soup.find('ul', id='section-icon-features-property')

feature_items = feature_section.find_all('li', class_='icon-feature')

In [85]:
details = {}

icon_to_attr = {
    'icon-stotal': 'total_area',
    'icon-scubierta': 'covered_area',
    'icon-ambiente': 'rooms',
    'icon-bano': 'bathrooms',
    'icon-cochera': 'parking_spaces',
    'icon-dormitorio': 'bedrooms',
    'icon-antiguedad': 'age'
}

for icon_class, attr_name in icon_to_attr.items():
    element = feature_section.find('i', class_=icon_class)
    if element and element.parent:
        value = element.parent.get_text(strip=True)
        # Extract only the numeric part
        numeric_value = re.search(r'\d+', value)
        if numeric_value:
            details[attr_name] = numeric_value.group()

In [93]:
# Find all script tags
script_tags = soup.find_all('script')

publisher_data = None

# Loop through all script tags
for script in script_tags:
    # Check if the script contains the publisher data
    if script.string and "'publisher':" in script.string:
        # Extract the JSON-like object
        match = re.search(r"'publisher'\s*:\s*(\{[^}]+\})", script.string)
        if match:
            # Get the matched group and replace single quotes with double quotes
            publisher_json = match.group(1).replace("'", '"')
            # Parse the JSON
            try:
                publisher_data = json.loads(publisher_json)
                break  # Exit the loop once we've found and parsed the data
            except json.JSONDecodeError:
                print("Error decoding JSON")

if publisher_data:
    print(f"Publisher name: {publisher_data.get('name')}")
    print(f"Publisher ID: {publisher_data.get('publisherId')}")
    print(f"Publisher URL: {publisher_data.get('url')}")
else:
    print("Publisher data not found")

Publisher name: MOHADEB PROPIEDADES
Publisher ID: 17032686
Publisher URL: /inmobiliarias/mohadeb-propiedades_17032686-inmuebles.html
